In [1]:
from pyspark.sql.functions import col
from pyspark.sql import *

def load_data():
    input_bucket = "s3://projectbucketpodguide/folderpodguide/"
    trip_path = 'podcastIDlist.csv'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Podcast List Count: ",trips.count())  
   
    return trips

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1615858487524_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
x = load_data()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Podcast List Count:  21320

In [5]:
mvv_list = x.selectExpr("name as name", "ID as _ID")
gg = [[(row._ID)] for row in mvv_list.collect()]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def getfeedURL(prmPodcasts):
    import json
    from urllib.request import urlopen
    import csv
    import re 
    #https://itunes.apple.com/lookup?id=270054094&country=US&media=podcast&entity=podcastEpisode&limit=1
    feedURLs = []
    for eachpodcast in prmPodcasts:
        country = 'us'#eachpodcast[0]
        pod_id = eachpodcast[0]
        dfeeds = []
        url = "https://itunes.apple.com/lookup?id=" + \
                str(pod_id) + \
                "&country=" + country + \
                "&media=podcast&entity=podcastEpisode&limit=0"
       # url = "https://itunes.apple.com/lookup?id=270054094&country=US&media=podcast&entity=podcastEpisode&limit=1"
        try:
            with urlopen(url) as data_file:    
                    reviewdata = json.load(data_file)
                    reviewdataresults = reviewdata["results"] 
                    feedUrl = reviewdataresults[0]["feedUrl"]

            dfeeds = [pod_id, country, feedUrl]        
        except Exception as err:
            print (err)
            pass
        
        feedURLs.append(dfeeds) 

    return feedURLs

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
qq = getfeedURL(gg)

In [67]:
kk = []  # This is to remove empty list.  Dont skip this step.  It wont convert to dataframe
for eachlist in qq:
    if len(eachlist) != 3:
        pass
    else:
        kk.append(eachlist)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
from pyspark.sql.types import *

deptColumns = ["podcastID","country","URL"]
deptDF = spark.createDataFrame(data=kk, schema = deptColumns)
deptDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- podcastID: string (nullable = true)
 |-- country: string (nullable = true)
 |-- URL: string (nullable = true)

In [74]:
deptDF.write.option("header","true").csv("s3://projectbucketpodguide/folderpodguide/output-feedURL1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…